#Natural Language Processing with Disaster Tweets
##Problem Definition
**Predict which Tweets are about real disasters and which ones are not**
##Data
**got this Dataset from Kaggle:**
https://www.kaggle.com/competitions/nlp-getting-started/overview
##Evaluation 
1. Classification Report
2. Confusion Matrix
3. Accuracy


In [ ]:
#Lets import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
import tensorflow_text as text

ModuleNotFoundError: ignored

In [ ]:
import tensorflow_hub as hub

In [ ]:
df=pd.read_csv('drive/MyDrive/Natural Language Processing with Disaster Tweets/train.csv')
df.head()

In [ ]:
df['text'][4000]

In [ ]:
df['target'].value_counts()

In [ ]:
X=df.drop('target',axis=1)
Y=df['target']

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
x, y = rus.fit_resample(X, Y)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x['text'],y, stratify=y,random_state=42)

In [ ]:
X_train.head()

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)


In [ ]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
154/154 [==============================] - 59s 313ms/step - loss: 0.6423 - accuracy: 0.6368 - precision: 0.6343 - recall: 0.6461
Epoch 2/100
154/154 [==============================] - 52s 338ms/step - loss: 0.6068 - accuracy: 0.6875 - precision: 0.6874 - recall: 0.6877
Epoch 3/100
154/154 [==============================] - 52s 336ms/step - loss: 0.5942 - accuracy: 0.6953 - precision: 0.6950 - recall: 0.6959
Epoch 4/100
154/154 [==============================] - 52s 340ms/step - loss: 0.5838 - accuracy: 0.7018 - precision: 0.7047 - recall: 0.6947
Epoch 5/100
154/154 [==============================] - 52s 336ms/step - loss: 0.5711 - accuracy: 0.7102 - precision: 0.7128 - recall: 0.7040
Epoch 6/100
154/154 [==============================] - 52s 338ms/step - loss: 0.5653 - accuracy: 0.7140 - precision: 0.7178 - recall: 0.7053
Epoch 7/100
154/154 [==============================] - 52s 340ms/step - loss: 0.5690 - accuracy: 0.7030 - precision: 0.7080 - recall: 0.6910
Epoch 8/100
1